<div style="font-family: 'Gen Jyuu Gothic Monospace Medium', 'Noto Sans TC', 'Inconsolata'; font-size: 150%; text-align: center; color: Plum;">
<br>

# <font color='Plum'><b>String Interning 字串留駐</b></font>
<br>
</div>
<div style="font-family: Inconsolata, 'Noto Sans TC'; font-size: 135%;">

* 本文附帶說明字串為何是immutable(不可變)。
* 原因先賣個關子，稍後各位自然了解...
</div>

In [5]:
import sys
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
print("InteractiveShell set.")
print(sys.version)

InteractiveShell set.
3.11.1 (main, Dec  7 2022, 01:11:34) [GCC 11.3.0]


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>物件比較</b></font>

* 對物件(變數)的比較，Python有兩個operators(運算子)：
    * 比較內容是否相等 ⮕ <span style="font-size: 150%; font-weight: 800; color: Gold;">==</span> (Comparison Operator)。
    * 比較是否同一物件 ⮕ <span style="font-size: 150%; font-weight: 800; color: Gold;">is</span> (Identity Operator)。

In [10]:
def compare_objs(*obj) -> None:
    comparison_operator = f'({obj[2]} == {obj[3]}) = {obj[0] == obj[1]}'
    identity_operator = f'({obj[2]} is {obj[3]}) = {obj[0] is obj[1]}'
    print(f'{comparison_operator:30}{identity_operator}')

print('compare_objs() defined.')

compare_objs() defined.


In [11]:
str1 = 'Alex';            str2 = 'Alex'
int1 = 23;                int2 = 23
float1 = 0.0;             float2 = 0.0
bool1 = True;             bool2 = True
none1 = None;             none2 = None
list1 = [23];             list2 = [23]
tuple1 = (23,);           tuple2 = (23,)
set1 = {33, 23};          set2 = {33, 23}
dict1 = {'id': 23};       dict2 = {'id': 23}

objs = ((str1, str2, 'str1', 'str2'),
        (int1, int2, 'int1', 'int2'), (float1, float2, 'float1', 'float2'),
        (bool1, bool2, 'bool1', 'bool2'), (none1, none2, 'none1', 'none2'),
        (list1, list2, 'list1', 'list2'), (tuple1, tuple2, 'tuple1', 'tuple2'),
        (set1, set2, 'set1', 'set2'), (dict1, dict2, 'dict1', 'dict2'))

for obj in objs:
    compare_objs(*obj)

(str1 == str2) = True         (str1 is str2) = True
(int1 == int2) = True         (int1 is int2) = True
(float1 == float2) = True     (float1 is float2) = False
(bool1 == bool2) = True       (bool1 is bool2) = True
(none1 == none2) = True       (none1 is none2) = True
(list1 == list2) = True       (list1 is list2) = False
(tuple1 == tuple2) = True     (tuple1 is tuple2) = False
(set1 == set2) = True         (set1 is set2) = False
(dict1 == dict2) = True       (dict1 is dict2) = False


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>本文聚焦於字串</b></font>

* 以 <span style="font-size: 150%; font-weight: 800; color: Gold;">is</span> 判斷str1和str2，結果如為 <span style="font-size: 150%; font-weight: 800; color: Gold;">True</span>，表示兩者為同一物件。

<div style="text-align:center"><img src="https://i.imgur.com/4ng2PiW.png" width="850"/></div>

* 以下用id()證明兩個字串變數，其值相同，就是同一物件。

In [16]:
name1 = 'Alex'   # name1和name2的值相同。
name2 = 'Alex'
print(f'{name1 = }\t\t{name2 = }')    # 兩者id()相同。
print(f'{id(name1) = }\n{id(name2) = }')
print(f'{name1 is name2 = }')   # 所以兩者指向同一reference。

name1 = 'Alex'		name2 = 'Alex'
id(name1) = 139847766079152
id(name2) = 139847766079152
name1 is name2 = True


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>What is String Interning?</b></font>

* 一些現代程式語言如Java, C#, Python, PHP, Ruby, Julia, ...等，如果多個字串變數(variables)內容相同，內部只會留下一份copy，這些不同名稱的變數全指向該copy的reference(記憶體位址)。意思是多個變數共用同一份字串，而非各有一份。
* 這個「<font color='tomato'>內容相同的字串只留一份</font>」機制，稱為<span style="color: Gold; font-size: 110%; font-weight: 800;">string interning</span>。intern這裡不是「實習」，而是「扣留、拘禁」的意思。本文譯作「<font color='tomato'><b>留駐</b></font>」，不過以下一律使用英文。
* 以下是2023-01-29筆者詢問ChatGPT的答案：
    > In Python, "string interning" refers to the process of creating a single, unique copy of a string and reusing it throughout the program. This is done to improve performance by reducing the amount of memory used and the number of object comparisons needed.
* <span style="color: Gold; font-size: 110%; font-weight: 800;">string interning</span>也可稱作`string caching`。ChatGPT的說明：
    > "String caching" is not a term that is commonly used in Python, but it can be considered similar to "integer caching" in that it refers to the process of reusing the same string objects in memory.<br><br>
    So, "string interning" and "string caching" are similar concept. Both refer to the process of reusing the same string objects in memory, however, "string interning" is more common term that is used in python.
* 所以下文還是用<span style="color: Gold; font-size: 110%; font-weight: 800;">string interning</span>。






<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>Why String Interning?</b></font>

* <span style="color: Gold; font-size: 110%; font-weight: 800;">string interning</span>的目的，是節省記憶體儲存空間，在比對時更有效率：
    * <font color='LightGreen'>節省空間</font>不言而喻，一份copy當然比多份copies少占用記憶體。
    * <font color='LightGreen'>更有效率</font>也不難理解，只核對一個reference，肯定快於逐個字元取出一一比較。
* Let’s assume that you have an application where a lot of string operations are happening. <span style="color: #FFC2C7;">If we were to use equality operator == for comparing long strings Python tries to compare it character by character</span>  and obviously it will take some time. But <span style="color: #FFE39F;">if these long strings can be interned then we know that they point to the same memory location.</span> In such a case we can use is keyword for comparing memory locations as it works much faster.
</div> 

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">
  
### <font color='DarkSalmon'><b>先打個岔：String Interning帶來的「副作用」</b></font>

* 正因為有interning機制，string就必須是<span style="font-size: 120%; font-weight: 800; color: Plum;">immutable</span>，即同一reference的字串，內容一旦設定即不可改變。
* 請想像：假如程式語言具有<span style="color: Gold; font-size: 110%; font-weight: 800;">string interning</span>機制，但字串卻設計成mutable(內容可變)，則修改某一變數內容，其他interning字串都會隨之更改。這種「連動效果」通常不是我們想要的。我們多半希望每個變數「相互獨立」而非「相互影響」。牽一髮不能動全身啊(註1)。

    ```python=
    'ABC'[1] = 'b'   # 字串是immutable，所以不能更改其中的字元。
    
    name = 'Alex'
    name = 'Mirror'  # 這時name指向另一個reference，所以沒有問題。
    ```
---
* 註1：Interning和mutability兩者的「因果關係」是否確定如此，筆者才疏學淺，無法參透，上述僅為個人臆測。

In [21]:
his_name = 'Liam'
her_name = 'Liam'  # 打錯了，應為'Lian'。

print(f'{his_name = }\t\t{her_name = }')
print(f'{id(his_name) = }\n{id(her_name) = }')
print(f'{(his_name is her_name) = }')

# her_name[-1] = 'n'    # 假如允許這樣修改her_name...
# print(f'{his_name = }\t\t{her_name = }')   # 那麼his_name也會跟著更改。

his_name = 'Liam'		her_name = 'Liam'
id(his_name) = 139847766214960
id(her_name) = 139847766214960
(his_name is her_name) = True


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>回歸正題</b></font>

* 「夫尺有所短，寸有所長，物有所不足，智有所不明，數有所不逮，神有所不通。」--屈原《楚辭．卜居》
* 「金無足赤，人無完人。」--宋．戴復古《寄興》
* 天下沒有白吃的午餐。--高希均名言
* 也沒有完美無缺的機制。Nothing is perfect!--Alex
<div style="text-align:left"><img src="https://i.imgur.com/R16nHnT.jpg" width="600"/></div>

---
* <span style="color: Gold; font-size: 110%; font-weight: 800;">string interning</span>固然可提高效能和節約空間，不過要去intern字串，以及維護interned字串的pool，也需要時間。
* 如果某個字串很長很長，又不會或絕少和其他字串比對，也許就不值得花額外時間去intern這個字串了。
* 所以，為了在<font color='LightGreen'>帶來的紅利</font>和<font color='LightGreen'>付出的代價</font>之間取得平衡，Python並不會對所有字串intern，而是訂出些「遊戲規則」，用來判斷某某字串要不要intern。
* 麻煩來了：這些遊戲規則，本身就繁瑣，而且不同的Python實作(註1)規則可能有別，即使是同一實作(如最普及的CPython)，也會因不同版本(註2)而有差異。這<span style="color: Gold; font-size: 110%; font-weight: 800;">string interning</span>機制真是「天威難測」呀。
---
* 註1：如CPython, IronPython, Jython, PyPy, MicroPython, ...
* 註2：例如CPython 3.7版以前使用[peephole optimization](https://github.com/python/cpython/blob/0f21fe6155227d11dc02bd3ef3b061de4ecea445/Python/peephole.c)方法決定是否intern字串，3.8版之後則改為[AST optimizer](https://github.com/python/cpython/blob/3.7/Python/ast_opt.c)。

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>String Interning規則例子</b></font>

* 以下規則肯定不周延，筆者也無意周延。
* 測試版本：CPython 3.10和3.11。


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>規則-1</b></font>

* 字串中含有非英文字母、數字或`'_'`(底線)者不intern。
* 例子：

In [35]:
# 字串含非英文字母、數字或`'_'`(底線)者不intern。
print()
name1 = 'Alex Van'     # 字串中有空白時不intern。
name2 = 'Alex Van'
print(f'{name1 = }\t\t\t{name2 = }')
print(f'{id(name1) = :<28}{id(name2) = }')
print(f'{(name1 is name2) = }\n')   # False

github1 = 'https://github.com/alexhtwen'   # 有':', '/', '.'等符號的字串不intern。
github2 = 'https://github.com/alexhtwen'
print(f'{github1 = :30}{github2 = :<0}')
print(f'{id(github1) = }\t\t{id(github2) = }')
print(f'{(github1 is github2) = }\n')   # False

greek1 = 'δθΩ'     # 這些希臘字母肯定不會intern。
greek2 = 'δθΩ'
print(f'{greek1 = }\t\t\t\t{greek2 = }')
print(f'{id(greek1) = }\t\t{id(greek2) = }')
print(f'{greek1 is greek2 = }\n')   # False

city1 = '澳門'    # 用肚子想都知道，中文鐵定不會自動intern。
city2 = '澳門'
print(f'{city1 = }\t\t\t\t{city2 = }')
print(f'{id(city1) = }\t\t{id(city2) = }')
print(f'{city1 is city2 = }\n')   # False

id1 = 'D103_a'   # 純英文字母、數字及底線組成的字串，終於intern了。
id2 = 'D103_a'
print(f'{id1 = }\t\t\t\t{id2 = }')
print(f'{id(id1) = :<20}\t\t{id(id2) = }')
print(f'{id1 is id2 = }\n')   # True
print()


name1 = 'Alex Van'			name2 = 'Alex Van'
id(name1) = 139847766226992             id(name2) = 139847766223152
(name1 is name2) = False

github1 = https://github.com/alexhtwen  github2 = https://github.com/alexhtwen
id(github1) = 139847766234800		id(github2) = 139847766234240
(github1 is github2) = False

greek1 = 'δθΩ'				greek2 = 'δθΩ'
id(greek1) = 139847766270736		id(greek2) = 139847766272368
greek1 is greek2 = False

city1 = '澳門'				city2 = '澳門'
id(city1) = 139847766266032		id(city2) = 139847766264688
city1 is city2 = False

id1 = 'D103_a'				id2 = 'D103_a'
id(id1) = 139847766322800     		id(id2) = 139847766322800
id1 is id2 = True




<div style="text-align:center"><img src="https://i.imgur.com/VmiMD9W.png" width="900"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>規則-2</b></font>

* 長度超過4,096個字元的字串不intern。
* CPython 3.7版本以前長度限制為20個字元。

In [40]:
str1 = 'A' * 4_096    # 連用5個4_096，何以不用變數？
str2 = 'A' * 4_096    # Alex是否功力大退？
str3 = 'A' * 4_096    # 當然不是。
str4 = 'A' * 4_096    # 請耐心看下去，自然明白。
str5 = 'A' * 4_096
print(f'{id(str1)=:<20}{id(str2)=:<20}{id(str3)=:<20}{id(str4)=:<20}{id(str5)=}')
print(f'{(str1 is str2 is str3 is str4 is str5) = }')

id(str1)=30980416            id(str2)=30984576            id(str3)=30598928            id(str4)=30603088            id(str5)=30990224
(str1 is str2 is str3 is str4 is str5) = False


<div style="text-align:center"><img src="https://i.imgur.com/NLIZX7y.png" width="900"/></div>

<div style="text-align:center"><img src="https://i.imgur.com/URaEamg.png" width="900"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>規則-3</b></font>

* 字串由string variable或string literal(註1)構成，有不同interning規則：
    * 下面的字串「會」觸發<span style="color: Gold; font-size: 110%; font-weight: 800;">string interning</span>:
        * 單一variable
        * 單一literal
        * literal + literal
    * 下面的字串「不會」觸發<span style="color: Gold; font-size: 110%; font-weight: 800;">string interning</span>(但有例外):
        * variable + variable
        * variable + literal
        * literal + variable
* 夠繁瑣了吧？
---
* 註1：variable是「變數」，literal為「定數」。例如在<span style="font-size: 150%; font-weight: 800; color: Gold;">`name = 'Alex'`</span>中，<span style="font-size: 150%; font-weight: 800; color: Gold;">`name`</span>是variable，<span style="font-size: 150%; font-weight: 800; color: Gold;">`'Alex'`</span>則是literal。

In [ ]:
first = 'first'
last = 'last'


In [160]:
name1 = 'Chat' + 'GPT'   # literal + literal
name2 = 'Chat' + 'GPT'
print(f'case 1: {(name1 is name2) = }')   # True

first = 'Chat'
last = 'GPT'

name1 = first + last   # variable + variable
name2 = first + last
print(f'case 2: {(name1 is name2) = }')   # False

name1 = first + 'GPT'   # variable + literal
name2 = first + 'GPT'
print(f'case 3: {(name1 is name2) = }')   # False

name1 = 'Chat' + last   # literal + variable
name2 = 'Chat' + last
print(f'case 4: {(name1 is name2) = }')   # False

case 1: (name1 is name2) = True
case 2: (name1 is name2) = False
case 3: (name1 is name2) = False
case 4: (name1 is name2) = False


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>Life is Short</b></font>

* 人生苦短，為何要花寶貴精力去記憶這些瑣碎規則？
* 況且規則還可能改來改去。
* 時間應該用於有生產力的地方，而不是拿來強記繁複又易變動的內部規則。
    <div style="text-align:left"><img src="https://i.imgur.com/jPb3Fic.jpg" width="500"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>有簡單的解決辦法嗎？</b></font>

* 請放心，Python永遠都會幫我們開扇窗的。
    <div style="text-align:center"><img src="https://i.imgur.com/6L0CEnl.jpg" width="700"/></div>    

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>內建的intern()函數</b></font>

* 上面所述的是Python自動提供的內隱式(implicit)<span style="color: Gold; font-size: 110%; font-weight: 800;">string interning</span>機制。
* 但此機制太過冗雜，建議不用記，也不要倚賴它來判斷字串是否為同一物件。
* 如果想有interning效果，請用Python內建的intern()函數外顯設定(intern a string explicitly)。

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

<div style="text-align:center"><img src="https://i.imgur.com/TGyUlKO.jpg" width="500"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>以intern()處理原規則-1</b></font>
* 原規則：字串中含有非英文字母、數字或`'_'`(底線)者不intern。
* 解決方法：外顯使用sys.intern()函數即有<span style="color: Gold; font-size: 110%; font-weight: 800;">string interning</span>效果。


In [39]:
# 字串含非英文字母、數字或`'_'`(底線)者不intern。
import sys

name1 = sys.intern('Alex Van')   # 本來有空白時不會intern。
name2 = sys.intern('Alex Van')   # 不自動就手動唄。
print(f'{name1 = }\t\t\t{name2 = }')
print(f'{id(name1) = }\t\t{id(name2) = }')
print(f'{(name1 is name2) = }\n')   # True

github1 = sys.intern('https://github.com/alexhtwen')
github2 = sys.intern('https://github.com/alexhtwen')
print(f'{github1 = :<30}{github2 = :<0}')
print(f'{id(github1) = }\t\t{id(github2) = }')
print(f'{(github1 is github2) = }\n')   # True

greek1 = sys.intern('δθΩ')
greek2 = sys.intern('δθΩ')
print(f'{greek1 = }\t\t\t\t{greek2 = }')
print(f'{id(greek1) = }\t\t{id(greek2) = }')
print(f'{greek1 is greek2 = }\n')   # True

city1 = sys.intern('澳門')
city2 = sys.intern('澳門')
print(f'{city1 = }\t\t\t\t{city2 = }')
print(f'{id(city1) = }\t\t{id(city2) = }')
print(f'{city1 is city2 = }\n')   # True

id1 = 'D103_a'   # 只有純英文字母、數字及底線組成的字串不必外顯itern()。
id2 = 'D103_a'
print(f'{id1 = }\t\t\t\t{id2 = }')
print(f'{id(id1) = }\t\t{id(id2) = }')
print(f'{id1 is id2 = }\n')   # True

name1 = 'Alex Van'			name2 = 'Alex Van'
id(name1) = 139847765808368		id(name2) = 139847765808368
(name1 is name2) = True

github1 = https://github.com/alexhtwen  github2 = https://github.com/alexhtwen
id(github1) = 139847766235440		id(github2) = 139847766235440
(github1 is github2) = True

greek1 = 'δθΩ'				greek2 = 'δθΩ'
id(greek1) = 139847765737072		id(greek2) = 139847765737072
greek1 is greek2 = True

city1 = '澳門'				city2 = '澳門'
id(city1) = 139847766269104		id(city2) = 139847766269104
city1 is city2 = True

id1 = 'D103_a'				id2 = 'D103_a'
id(id1) = 139847766322800		id(id2) = 139847766322800
id1 is id2 = True



In [2]:
import sys

name1 = sys.intern(('Alex', ))
name2 = sys.intern(('Alex', ))

name1 is name2

TypeError: intern() argument must be str, not tuple

<div style="text-align:center"><img src="https://i.imgur.com/JVDSndE.png" width="900"/></div>


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>以intern()處理原規則-2</b></font>
* 原規則：長度超過4,096個字元的字串不intern。
* 解決方法：外顯使用sys.intern()函數即有<span style="color: Gold; font-size: 110%; font-weight: 800;">string interning</span>效果。

In [48]:
# 字串直接賦值
str1 = 'A' * 4_097
str2 = 'A' * 4_097
print(f'{id(str1) = :<20}{id(str2) = }')
print(f'{(str1 is str2) = }\n')   # False
# -----------------
# 使用intern()函數
from sys import intern

str1 = intern('A' * 4_097)
str2 = intern('A' * 4_097)
print(f'{id(str1) = :<20}{id(str2) = }')
print(f'{(str1 is str2) = }')   # True

id(str1) = 59406032            id(str2) = 59410192
(str1 is str2) = False

id(str1) = 59400592            id(str2) = 59400592
(str1 is str2) = True


<div style="text-align:center"><img src="https://i.imgur.com/36C4Ajn.png" width="700"/></div>


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>以intern()處理原規則-3</b></font>
* 原規則：字串由variable組合而成者不intern。
* 解決方法：外顯使用sys.intern()函數即有<span style="color: Gold; font-size: 110%; font-weight: 800;">string interning</span>效果。

In [46]:
# 字串直接賦值
first = 'first'
last = 'last'
name1 = first + 'last'
name2 = 'first' + 'last'
print(f'{name1 = }\t\t{name2 = }')
print(f'{id(name1) = :<20}{id(name2) = }')
print(f'{(name1 is name2) = }\n')
# -----------------
# 使用intern()函數
from sys import intern

first = 'first'
last = 'last'
name1 = intern(first + 'last')
name2 = intern('first' + 'last')
print(f'{name1 = }\t\t{name2 = }')
print(f'{id(name1) = :<20}{id(name2) = }')
print(f'{(name1 is name2) = }')

name1 = 'firstlast'		name2 = 'firstlast'
id(name1) = 139847766315760     id(name2) = 139847766317488
(name1 is name2) = False

name1 = 'firstlast'		name2 = 'firstlast'
id(name1) = 139847766317488     id(name2) = 139847766317488
(name1 is name2) = True


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>效能測試</b></font>
* 以下是有無interning的效能比較測試。

In [54]:
from sys import intern
import random as rd
import time
from string import ascii_letters

str_lens = (100, 10_000, 5_000_000)
loops = (100, 10_000, 1_000_000)

for str_len in str_lens:
    rand_str = ''.join(rd.choices(ascii_letters + '人生苦短', k=str_len))
    str1 = ' Α ' + rand_str + ' Ω '
    str2 = ' Α ' + rand_str + ' Ω '
    str3 = intern(rand_str + ' The End. ')
    str4 = intern(rand_str + ' The End. ')

    for loop in loops:
        time1 = time.time()
        for _ in range(loop):
            is_equal_sans_interning = (str1 == str2)
        time2 = time.time()
        duration_sans_interning = time2 - time1
        print(f'{str_len = :<12}{loop = :<12}')
        print(f'{duration_sans_interning = }')

        # ================================
        time3 = time.time()
        for _ in range(loop):
            is_equal_with_interning = (str3 == str4)
        time4 = time.time()
        duration_with_interning = time4 - time3
        print(f'{duration_with_interning = }')
        # print()
        print(f'{(duration_with_interning < duration_sans_interning) = }')
        print(f'{(duration_with_interning / duration_sans_interning) = :.10f}')
        print('------')
    print('===========================')

str_len = 100         loop = 100         
duration_sans_interning = 1.3828277587890625e-05
duration_with_interning = 1.9073486328125e-05
(duration_with_interning < duration_sans_interning) = False
(duration_with_interning / duration_sans_interning) = 1.3793103448
------
str_len = 100         loop = 10000       
duration_sans_interning = 0.0009982585906982422
duration_with_interning = 0.0007781982421875
(duration_with_interning < duration_sans_interning) = True
(duration_with_interning / duration_sans_interning) = 0.7795557679
------
str_len = 100         loop = 1000000     
duration_sans_interning = 0.10639715194702148
duration_with_interning = 0.10081720352172852
(duration_with_interning < duration_sans_interning) = True
(duration_with_interning / duration_sans_interning) = 0.9475554719
------
str_len = 10000       loop = 100         
duration_sans_interning = 3.266334533691406e-05
duration_with_interning = 1.0013580322265625e-05
(duration_with_interning < duration_sans_interning) = T

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>效能比較發現</b></font>

* 比對的字串較短時，interning與否效能沒啥差別，甚至有時不intern反而比較快。字串要到很長時，interning的功用才陸續顯示出來。
* 迭代次數越多，interning效果越顯著。

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>結論</b></font>

* 比對兩物件(變數)的「<span style="font-size: 120%; font-weight: 500; color: Tomato;">內容</span>」是否相等，要用<span style="font-size: 135%; font-weight: 700; color: Gold;">==</span>，不要用<span style="font-size: 135%; font-weight: 700; color: Gold;">is</span>。
* 判斷是否為<span style="font-size: 120%; font-weight: 500; color: Tomato;">None</span>(註1)則用<span style="font-size: 135%; font-weight: 700; color: Gold;">is</span>。<span style="font-size: 135%; font-weight: 700; color: Gold;">==</span>本來也可以，不過[PEP 8](https://peps.python.org/pep-0008/)規範用<span style="font-size: 135%; font-weight: 700; color: Gold;">is</span>。
* PEP 8截圖：
    <div style="text-align:center"><img src="https://i.imgur.com/jMJ3Ziq.png" width="900"/></div>
---
* 註1：因為None是singleton，整個Python程式只有一份。


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>String Interning到此結束...</b></font>

<div style="text-align:center"><img src="https://i.imgur.com/cV6EtSL.png" width="250"/></div>

<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>域外誌：Integer Interning(caching)</b></font>

* Python的integer原來也具備<span style="color: Gold; font-size: 110%; font-weight: 800;">interning</span>機制。

> "Integer caching" and "integer interning" are similar concepts, but they are not exactly the same.<br><br>
"Integer caching" refers to the process of reusing the same small integer objects in memory, so that multiple variables can refer to the same object. This is a memory optimization technique that is used to save memory space. It is implemented in Python and other programming languages.<br><br>
"Integer interning" is a more general concept, which refers to the process of creating a single, unique copy of a string or integer, and then reusing it throughout the program. It is used to improve performance by reducing the amount of memory used and the number of object comparisons needed. It is also implemented in many programming languages, including Python.<br><br>
In Python, the terms "integer caching" and "integer interning" are often used interchangeably, as both refer to the process of reusing small integers in memory. However, "integer caching" is more specific to Python, while "integer interning" is a more general concept that applies to other languages too.

* 不過implicit integer interning和string interning相似，也有些瑣碎規則：


<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

### <font color='DarkSalmon'><b>上下限</b></font>
    * -5 ~ 256的整數會自動intern。
    * -6以下或257以上則不會。
* -5 ~ 256的整數只有一份，故稱作singleton objects。

In [76]:
def print_nums(num1: int | float, num2: int | float) -> None:
    print(f'{num1 = :<24}{num2 = }')
    print(f'{id(num1) = :<20}{id(num2) = }')
    print(f'{(num1 is num2) = }\n')

In [74]:
# implicit的整數interning範圍為-5 ~ 256。
print()
num1 = 5
num2 = 5
print_nums(num1, num2)

num1 = 100
num2 = 100
print_nums(num1, num2)

num1 = 256
num2 = 256
print_nums(num1, num2)

num1 = -6
num2 = -6
print_nums(num1, num2)

num1 = 257
num2 = 257
print_nums(num1, num2)


num1 = 5                       num2 = 5
id(num1) = 10742664            id(num2) = 10742664
(num1 is num2) = True

num1 = 100                     num2 = 100
id(num1) = 10745704            id(num2) = 10745704
(num1 is num2) = True

num1 = 256                     num2 = 256
id(num1) = 10750696            id(num2) = 10750696
(num1 is num2) = True

num1 = -6                      num2 = -6
id(num1) = 139847767360752     id(num2) = 139847767359920
(num1 is num2) = False

num1 = 257                     num2 = 257
id(num1) = 139847767360624     id(num2) = 139847767360560
(num1 is num2) = False



In [88]:
# implicit的整數interning範圍為-5 ~ 256。
print()
num1 = 5
num2 = 5
print(f'{num1 is num2 = }')   # True

num1 = 100
num2 = 100
print(f'{num1 is num2 = }')   # True

num1 = 256
num2 = 256
print(f'{num1 is num2 = }')   # True

num1 = -6
num2 = -6
print(f'{num1 is num2 = }')   # False

num1 = 257
num2 = 257
print(f'{num1 is num2 = }')   # False


num1 is num2 = True
num1 is num2 = True
num1 is num2 = True
num1 is num2 = False
num1 is num2 = False


In [75]:
numbers = [-5, 100, 256, -6, 257]
print()
for number in numbers:
    num1 = number
    num2 = number
    print_nums(num1, num2)


num1 = -5                      num2 = -5
id(num1) = 10742344            id(num2) = 10742344
(num1 is num2) = True

num1 = 100                     num2 = 100
id(num1) = 10745704            id(num2) = 10745704
(num1 is num2) = True

num1 = 256                     num2 = 256
id(num1) = 10750696            id(num2) = 10750696
(num1 is num2) = True

num1 = -6                      num2 = -6
id(num1) = 139847767372752     id(num2) = 139847767372752
(num1 is num2) = True

num1 = 257                     num2 = 257
id(num1) = 139847767360784     id(num2) = 139847767360784
(num1 is num2) = True



In [93]:
numbers = [-5, 100, 256, -6, 999**999]
for number in numbers:
    num1 = number
    num2 = number
    print(f'{number = }\n{num1 is num2 = }')

number = -5
num1 is num2 = True
number = 100
num1 is num2 = True
number = 256
num1 is num2 = True
number = -6
num1 is num2 = True
number = 368063488259223267894700840060521865838338232037353204655959621437025609300472231530103873614505175218691345257589896391130393189447969771645832382192366076536631132001776175977932178658703660778465765811830827876982014124022948671975678131724958064427949902810498973271030787716781467419524180040734398996952930832508934116945966120176735120823151959779536852290090377452502236990839453416790640456116471139751546750048602189291028640970574762600185950226138244530187489211615864021135312077912018844630780307462205252807737757672094320692373101032517459518497524015120165166724189816766397247824175394802028228160027100623998873667435799073054618906855460488351426611310634023489044291860510352301912426608488807462312126590206830413782664554260411266378866626653755763627796569082931785645600816236891168141774993267488171702172191072731069216881668294625679

In [148]:
num1 = 256
num2 = 256
print(f'case 1: {num1 is num2 = }')   # True

num1 = 256 + 0
num2 = 256 + 0
print(f'case 2: {num1 is num2 = }')   # True

num1 = 257
num2 = 257
print(f'case 3: {num1 is num2 = }')   # False

num1 = 257 + 0
num2 = 257 + 0
print(f'case 4: {num1 is num2 = }')   # False

number = 256
num1 = number
num2 = number
print(f'case 5: {num1 is num2 = }')   # True

num1 = number + 0
num2 = number + 0
print(f'case 6: {num1 is num2 = }')   # True

number = 257
num1 = number
num2 = number
print(f'case 7: {num1 is num2 = }')   # True ??

num1 = number + 0
num2 = number + 0
print(f'case 8: {num1 is num2 = }')   # False ??

str1 = 'a' + '1'
str2 = 'a' + '1'
print(f'case 9: {str1 is str2 = }')   # True

case 1: num1 is num2 = True
case 2: num1 is num2 = True
case 3: num1 is num2 = False
case 4: num1 is num2 = False
case 5: num1 is num2 = True
case 6: num1 is num2 = True
case 7: num1 is num2 = True
case 8: num1 is num2 = False
case 9: str1 is str2 = True


In [155]:
import copy
x = ({1: 'a', 2: 'b', 3: 'c'},)
y = copy.copy(x)
x[0][1] = 'd'
print(f'{x = }')
print(f'{y = }')
2023-1946

x = ({1: 'd', 2: 'b', 3: 'c'},)
y = ({1: 'd', 2: 'b', 3: 'c'},)


77

In [ ]:


a = [1, 2, 5, 9]
b = copy.copy(a)
b[2] = 999
a
b


In [ ]:

num1 = number
num2 = number
print(f'{num1 is num2 = }')

num1 = number + 0
num2 = number + 0
print(f'{num1 is num2 = }')

# name1 = 'Alex'
# name2 = 'ander'
# str1 = 'name1' + 'name2'
# str2 = 'name1' + 'name2'
# print(f'{str1 is str2 = }')

In [80]:
print()
num1 = 3.1416
num2 = 3.1416
print_nums(num1, num2)

num1 = 0.0
num2 = 0.0
print_nums(num1, num2)

num1 = -1.5
num2 = -1.5
print_nums(num1, num2)


num1 = 3.1416                  num2 = 3.1416
id(num1) = 139847767360080     id(num2) = 139847767360752
(num1 is num2) = False

num1 = 0.0                     num2 = 0.0
id(num1) = 139847767360560     id(num2) = 139847767360720
(num1 is num2) = False

num1 = -1.5                    num2 = -1.5
id(num1) = 139847767363024     id(num2) = 139847767364720
(num1 is num2) = False



<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">



<div style="font-family: 'Inconsolata', 'Noto Sans TC'; font-size: 135%;">

## <font color='SteelBlue'><b>外顯Intern整數</b></font>

* 目前筆者不知道如何外顯intern整數。
* 所以暫時無解。

In [33]:
from sys import intern

num1 = intern(256)    # implicit的整數interning上限為256。
num2 = intern(256)

# num1 = 257
# num2 = 257
print(f'{num1 = }\t\t{num2 = }')
print(f'{id(num1) = :<20}{id(num2) = }')
print(f'{(num1 is num2) = }')

print(123)

num1 = 257		num2 = 257
id(num1) = 140062901148080     id(num2) = 140062901148112
(num1 is num2) = False
123


<div style="font-family: 'Gen Jyuu Gothic Monospace Medium', 'Noto Sans TC'; font-size: 200%;  text-align: center; color: Plum;">

<br><br>
# The End